### MODELs
|No|Model Name|Deploy link|Tested|Fine-tune|
|---|---|---|---|---|
|1|[CohereForAI/aya-23-8B](https://huggingface.co/CohereForAI/aya-23-8B)|https://huggingface.co/spaces/CohereForAI/aya-23-8b|Tested|---|
|2|[OrionStarAI/Orion-14B-Chat](https://huggingface.co/OrionStarAI/Orion-14B-Chat)|https://huggingface.co/spaces/OrionStarAI/Orion-14B-App-Demo-EN|Tested|---|
|3|[Qwen/Qwen2.5-Coder-7B-Instruct](https://huggingface.co/Qwen/Qwen2.5-Coder-7B-Instruct)|https://huggingface.co/spaces/Qwen/Qwen2.5-Coder-7B-Instruct|Tested|---|
|4|[meta-llama/Meta-Llama-3.1-8B-Instruct](https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct)|https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct|---|---|

### CohereForAI/aya-23-8B

In [1]:
import warnings
warnings.filterwarnings("ignore")

import random
import torch
import pandas as pd

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "CohereForAI/aya-23-8B"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             device_map="auto",
                                             torch_dtype=torch.bfloat16,)

# Format message with the command-r-plus chat template
messages = [{"role": "user", "content": "Anneme onu ne kadar sevdiğimi anlatan bir mektup yaz"}]
input_ids = tokenizer.apply_chat_template(messages, tokenize=True,
                                          add_generation_prompt=True,
                                          return_tensors="pt")
## <BOS_TOKEN><|START_OF_TURN_TOKEN|><|USER_TOKEN|>Anneme onu ne kadar sevdiğimi anlatan bir mektup yaz<|END_OF_TURN_TOKEN|><|START_OF_TURN_TOKEN|><|CHATBOT_TOKEN|>

gen_tokens = model.generate(
    input_ids, 
    max_new_tokens=50, 
    do_sample=True, 
    temperature=0.3,
    )

gen_text = tokenizer.decode(gen_tokens[0],skip_special_tokens=True)
print(gen_text)

RuntimeError: Failed to import transformers.models.cohere.modeling_cohere because of the following error (look up to see its traceback):
libtorch_cuda_cu.so: cannot open shared object file: No such file or directory

In [6]:
def get_completion(sentence:str):

    messages = [{"role": "user", "content": sentence}]

    input_ids = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt")
    
    gen_tokens = model.generate(
        input_ids, 
        max_new_tokens=50, 
        do_sample=True, 
        temperature=0.3,
    )

    gen_text = tokenizer.decode(gen_tokens[0],skip_special_tokens=True)
    return gen_text

In [7]:
get_completion("こんにちは、お会いできて嬉しいです")

'<|START_OF_TURN_TOKEN|><|USER_TOKEN|>こんにちは、お会いできて嬉しいです<|START_OF_TURN_TOKEN|><|CHATBOT_TOKEN|>こんにちは！私もお会いできて嬉しいです。何かお手伝いできることはありますか？'

In [10]:
# Load and preprocess dataset
QA_set = load_dataset("locchuong/Mainframe-QA-en-ja-300")
QA_set

DatasetDict({
    train: Dataset({
        features: ['id', 'prompt', 'question', 'answer', 'anwser_ja', 'question_ja'],
        num_rows: 300
    })
})

In [13]:
def add_completion(example):
    return {"completion":get_completion(example["question_ja"])}

In [ ]:
QA_set = QA_set.map(add_completion)

In [12]:
total_sample = QA_set['train'].num_rows
random_idx = random.choice(range(total_sample))
question = QA_set['train'][random_idx]['question_ja']
answer = QA_set['train'][random_idx]['anwser_ja']
completion = get_completion(question)

print("Question:\n",question)
print("Answer:\n",answer)
print("Completion:\n",completion)

Question:
 COBOLでナストされたプログラムを定義するための合成とは何ですか?
Answer:
 COBOL でナストされたプログラムを定義するには、「PROGRAM-ID」の声明を使用し、「END PROGRAM」の声明に続きます。
Completion:
 <|START_OF_TURN_TOKEN|><|USER_TOKEN|>COBOLでナストされたプログラムを定義するための合成とは何ですか?<|START_OF_TURN_TOKEN|><|CHATBOT_TOKEN|>合成とは、COBOL プログラミング言語における重要な概念であり、ナストされたプログラム定義の作成に使用されます。合成とは、複数の COBOL プログラムを 1 つの論理単位として組み合わせるプロセスです。これにより、


### Qwen/Qwen2.5-Coder-7B-Instruct

In [1]:
import warnings
warnings.filterwarnings("ignore")

import random
import torch
import pandas as pd

from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-Coder-7B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float32,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

prompt = "write a quick sort algorithm."
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Certainly! QuickSort is a highly efficient sorting algorithm that uses a divide-and-conquer approach to sort elements. Here's a Python implementation of the QuickSort algorithm:

```python
def quicksort(arr):
    if len(arr) <= 1:
        return arr
    else:
        pivot = arr[len(arr) // 2]
        left = [x for x in arr if x < pivot]
        middle = [x for x in arr if x == pivot]
        right = [x for x in arr if x > pivot]
        return quicksort(left) + middle + quicksort(right)

# Example usage:
arr = [3, 6, 8, 10, 1, 2, 1]
sorted_arr = quicksort(arr)
print("Sorted array:", sorted_arr)
```

### Explanation:
1. **Base Case**: If the array has 0 or 1 elements, it is already sorted, so we return it as is.
2. **Pivot Selection**: We choose the pivot element from the array. In this example, we select the middle element.
3. **Partitioning**: We create three sub-arrays:
   - `left`: All elements less than the pivot.
   - `middle`: All elements equal to the pivot.
   - `right`: All e

In [3]:
def get_completion(sentence:str):
    messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": sentence}
    ]
    
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
    
    generated_ids = model.generate(**model_inputs, max_new_tokens=100)
    
    generated_ids = [
        output_ids[len(input_ids):] for input_ids,
        output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    return tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [7]:
get_completion("こんにちは、お会いできて嬉しいです")

'こんにちは！お会いできましたら嬉しいです。何かお手伝いできることがありましたら、遠慮なくお知らせください。'

In [8]:
# Load and preprocess dataset
QA_set = load_dataset("locchuong/Mainframe-QA-en-ja-300")
QA_set

DatasetDict({
    train: Dataset({
        features: ['id', 'prompt', 'question', 'answer', 'anwser_ja', 'question_ja'],
        num_rows: 300
    })
})

In [11]:
total_sample = QA_set['train'].num_rows
random_idx = random.choice(range(total_sample))
question = QA_set['train'][random_idx]['question_ja']
answer = QA_set['train'][random_idx]['anwser_ja']
completion = get_completion(question)

print("Question:\n",question)
print("Answer:\n",answer)
print("Completion:\n",completion)

Question:
 どのタイプのコンピュータが科学的計算やシミュレーションに最適ですか?
Answer:
 スーパーコンピュータは、高度なパフォーマンス能力と高度なアーキテクチャにより、科学的計算とシミュレーションに最適です 彼らは高速と正確さで大量のデータを処理することができ、複雑な数学的計算とシミュレーションに最適です。
Completion:
 科学的計算やシミュレーションに最適なコンピュータは、高性能・高度なパフォーマンスを必要とする専用の「スーパーコンピュータ」です。

スーパーコンピュータは、複数のプロセッサ（CPU）とメモリを組み合わせた高機能なシステムで、一般的なパソコンでは見つけることができません。また、高速ネットワークや大容量のスト


In [ ]:
def add_completion(example):
    return {"completion":get_completion(example["question_ja"])}

### OrionStarAI/Orion-14B-Chat

In [1]:
import warnings
warnings.filterwarnings("ignore")

import random
import torch
import pandas as pd

from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

from transformers.generation.utils import GenerationConfig

tokenizer = AutoTokenizer.from_pretrained("OrionStarAI/Orion-14B-Chat", use_fast=False, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("OrionStarAI/Orion-14B-Chat", device_map="auto",
                                             torch_dtype=torch.bfloat16, trust_remote_code=True)

model.generation_config = GenerationConfig.from_pretrained("OrionStarAI/Orion-14B-Chat")
messages = [{"role": "user", "content": "Hello, what is your name? "}]
response = model.chat(tokenizer, messages, streaming=False)
print(response)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Hello! I am an AI language model created by OrionStar. My name is ChatGPT, but you can call me GPT for short. How can I assist you today?


In [2]:
def get_completion(sentence:str):
    messages = [{"role": "user", "content": sentence}]
    return model.chat(tokenizer, messages, streaming=False)

In [3]:
get_completion("こんにちは、お会いできて嬉しいです")

'こんにちは!あなたにお会いできることを楽しみにしています。どのような質問や会話のきっかけをお望みですか？'

In [4]:
# Load and preprocess dataset
QA_set = load_dataset("locchuong/Mainframe-QA-en-ja-300")
QA_set

DatasetDict({
    train: Dataset({
        features: ['id', 'prompt', 'question', 'answer', 'anwser_ja', 'question_ja'],
        num_rows: 300
    })
})

In [5]:
total_sample = QA_set['train'].num_rows
random_idx = random.choice(range(total_sample))
question = QA_set['train'][random_idx]['question_ja']
answer = QA_set['train'][random_idx]['anwser_ja']
completion = get_completion(question)

print("Question:\n",question)
print("Answer:\n",answer)
print("Completion:\n",completion)

Question:
 メインフレームテストにおけるテストケースの役割は何ですか?
Answer:
 メインフレームテストにおけるテストケースは、ステップ、入力、予想結果、およびテストを実行するために必要な追加情報を定義する詳細で繰り返す手順です。
Completion:
 メインフレームテストにおけるテストケースは、システムが要件を満たし、ユーザーの期待通りに動作することを確認するために使用されます。テストケースは、システムの特定の側面をテストするために設計され、コードが正確で信頼性があり、意図した通りに動作していることを検証します。また、バグやその他の問題を発見するのにも役立ち、システムの全体的な品質を確保するために重要です。


In [6]:
def add_completion(example):
    return {"completion":get_completion(example["question_ja"])}

In [7]:
QA_set = QA_set.map(add_completion)

Parameter 'function'=<function add_completion at 0x792d1b29ba60> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [8]:
# Convert the dataset to a pandas DataFrame (for train split)
df_train = pd.DataFrame(QA_set['train'])

# Save the DataFrame to a CSV file
df_train.to_csv('Orion-14B-Chat.csv', index=False)

print("Dataset successfully exported to CSV!")

Dataset successfully exported to CSV!


In [9]:
df_train.head()

,id,prompt,question,answer,anwser_ja,question_ja,completion
0,2276,"As a supportive AI assistant, you've been give...",What is the role of the line printer in a main...,The line printer is responsible for printing h...,ラインプリンターは、メインフレームからハードコピーの出力を印刷する責任があり、通常は長期保存...,メインフレームユーザーインターフェイスにおけるラインプリンターの役割は何ですか?,ラインプリンターは、メインフレームコンピュータのユーザーインターフェースで使用されるデバイス...
1,974,"As a helpful AI assistant, you've received a q...",What is the syntax in COBOL to display the cur...,"To display the current system time in COBOL, y...",COBOL で現在のシステム時間を表示するには、CURRENT-TIME 特別記録で EXT...,COBOL で現在のシステムタイムを表示するための合成は何ですか?,COBOL で現在のシステムタイムを表示するためには、SYST-TIMEを取得する必要があります。
2,1153,"In your role as a supportive AI assistant, you...",What was the role of COBOL 2002?,"COBOL 2002, officially known as ISO/IEC 1989:2...",COBOL 2002は、正式にISO/IEC 1989:2002として知られ、第3のCOBO...,COBOL 2002の役割は?,COBOL 2002は、米国コボルプログラミング言語協議会によって1999年に発表されたCO...
3,751,"In your role as a supportive AI assistant, you...",What is the role of the FILE SECTION?,The File Section is used to define the file la...,ファイルセクションは、ファイルの配置とファイルの状態、レコード形式、レコードキーなどの関連情...,ファイルセクションの役割は?,ファイルセクションは、プログラムのソースコードやデータ、ライブラリなどをファイルに保存するた...
4,1826,You are a helpful AI assistant. You have been ...,What is the role of automation in modern mainf...,Automation plays a crucial role in modern main...,自動化は現代のメインフレームメンテナンスにおいて重要な役割を果たし、メンテナンス作業に必要な...,現代のメインフレームメンテナンスにおける自動化の役割は何ですか?,現代のメインフレームメンテナンスにおいて自動化は重要な役割を果たしています。手動で行う必要が...


### meta-llama/Meta-Llama-3.1-8B-Instruct 	

In [1]:
import warnings
warnings.filterwarnings("ignore")

import random
import torch
import pandas as pd

import transformers


from datasets import load_dataset


model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.float32},
    device_map="auto",
)

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1]['content'])


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Yer lookin' fer a swashbucklin' introduction, eh? Well, matey, I be Captain Chat, the scurvy dog o' a chatbot! Me and me trusty keyboard be here to guide ye through the seven seas o' knowledge, answerin' yer questions and servin' up a healthy dose o' pirate-themed fun! So hoist the sails and set course fer adventure, me hearty!


In [2]:
def get_completion(sentence:str):

    messages = [
    {"role": "user", "content": sentence},
    ]
    
    outputs = pipeline(
    messages,
    max_new_tokens=100,
    )
    return outputs[0]["generated_text"][-1]['content']

In [3]:
get_completion("こんにちは、お会いできて嬉しいです")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'お互い会うことができて嬉しいです。どういたしまして。何についてお話したいですか？'

In [6]:
# Load and preprocess dataset
QA_set = load_dataset("locchuong/Mainframe-QA-en-ja-300")
QA_set

DatasetDict({
    train: Dataset({
        features: ['id', 'prompt', 'question', 'answer', 'anwser_ja', 'question_ja'],
        num_rows: 300
    })
})

In [7]:
total_sample = QA_set['train'].num_rows
random_idx = random.choice(range(total_sample))
question = QA_set['train'][random_idx]['question_ja']
answer = QA_set['train'][random_idx]['anwser_ja']
completion = get_completion(question)

print("Question:\n",question)
print("Answer:\n",answer)
print("Completion:\n",completion)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question:
 COBOLエラー処理におけるエラーメッセージの役割は何ですか?
Answer:
 COBOLエラー処理におけるエラーメッセージは、エラーの原因を特定し、問題解決に役立つ情報を提供する上で重要な役割を果たします。
Completion:
 COBOLエラー処理におけるエラーメッセージの役割は、システムがエラーが発生した際に、ユーザーに起こったエラーの詳細を伝えるものです。

エラーメッセージは、システムがエラーを検出すると、ユーザーに表示されるメッセージです。これにより、ユーザーはエラーが発生した原因を理解し、適切な


In [8]:
def add_completion(example):
    return {"completion":get_completion(example["question_ja"])}

In [ ]:
QA_set = QA_set.map(add_completion)

In [ ]:
# Convert the dataset to a pandas DataFrame (for train split)
df_train = pd.DataFrame(QA_set['train'])

# Save the DataFrame to a CSV file
df_train.to_csv('Meta-Llama-3.1-8B-Instruct.csv', index=False)

print("Dataset successfully exported to CSV!")